In [4]:
!pip install -q kaggle

In [5]:
from google.colab import files

# Upload the kaggle.json file that you downloaded
uploaded = files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Saving kaggle.json to kaggle.json
100% 2.29G/2.29G [01:25<00:00, 30.5MB/s]
100% 2.29G/2.29G [01:25<00:00, 28.7MB/s]


In [ ]:
!mkdir project
!unzip chest-xray-pneumonia.zip -d project

In [7]:
import numpy as np # linear algebra
import pandas as pd
import os

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
from tensorflow import keras
from keras.models import Model , load_model
from keras.applications.vgg16 import VGG16
from keras.layers import  Dense, Flatten
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import cv2

In [10]:
IMAGE_SIZE = [224, 224]

path_train = '/content/project/chest_xray/chest_xray/train'
path_validation = '/content/project/chest_xray/chest_xray/val'

In [11]:
vgg_model = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


58889256/58889256 [==============================] - 3s 0us/step


In [12]:
for layer in vgg_model.layers:
    layer.trainable = False

In [13]:
folders = glob('/content/project/chest_xray/chest_xray/train/*')
x = Flatten()(vgg_model.output)


In [14]:
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg_model.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [17]:
train_df = '/content/project/chest_xray/chest_xray/train'
test_df = '/content/project/chest_xray/chest_xray/test'

In [18]:
training_data = train_datagen.flow_from_directory('/content/project/chest_xray/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')




testing_data = test_datagen.flow_from_directory('/content/project/chest_xray/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [19]:
history = model.fit_generator(
  training_data,
  validation_data=testing_data,
  epochs=4,
  steps_per_epoch=len(training_data),
  validation_steps=len(testing_data),
)

Epoch 1/4
522/522 [==============================] - 126s 231ms/step - loss: 0.1978 - accuracy: 0.9273 - val_loss: 0.3116 - val_accuracy: 0.9215
Epoch 2/4
522/522 [==============================] - 115s 220ms/step - loss: 0.1508 - accuracy: 0.9521 - val_loss: 0.4279 - val_accuracy: 0.9006
Epoch 3/4
522/522 [==============================] - 118s 227ms/step - loss: 0.1527 - accuracy: 0.9548 - val_loss: 0.8827 - val_accuracy: 0.8462
Epoch 4/4
522/522 [==============================] - 114s 219ms/step - loss: 0.1076 - accuracy: 0.9649 - val_loss: 0.4452 - val_accuracy: 0.9247


In [20]:
score = model.evaluate(testing_data, steps = len(test_df)/10, verbose = 0)
print('loss:', score[0])
print('accuracy:', score[1])

loss: 0.10723571479320526
accuracy: 0.9599999785423279


In [21]:
model.save('Pneumonia.h5')

In [24]:
model=load_model('Pneumonia.h5')

In [23]:
img=image.load_img('/content/project/chest_xray/chest_xray/val/PNEUMONIA/person1951_bacteria_4882.jpeg')
resized_img = cv2.resize(np.array(img), (224, 224))

In [25]:
i = image.img_to_array(resized_img)

In [26]:
i = np.expand_dims(i, axis=0)

In [27]:
img_data = preprocess_input(i)

In [28]:
classes = model.predict(img_data)
print(classes)

1/1 [==============================] - 1s 1s/step
[[0. 1.]]


In [29]:
result=int(classes[0][0])

In [30]:
if result == 0:
    print("Person is Affected By PNEUMONIA")
else:
    print("Result is Normal")

Person is Affected By PNEUMONIA
